In [4]:
from ultralytics import YOLO
import pandas as pd
import os
import cv2

# Video to keypoints csv

# GENERATE keypoints here
model = YOLO("yolo11n-pose.pt")
model.track("./fall_videos/mc_videos_test/chute01/cam1.avi", show=True)



WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1562) C:\Users\sonnpm\OneDrive - UTS\Desktop\UTS\Semester 3\DL\Assignment\ass 3\fall_videos\mc_videos_test\chute01\cam1.avi: 448x640 1 person, 35.0ms
video 1/1 (frame 2/1562) C:\Users\sonnpm\OneDrive - UTS\Desktop\UTS\Semester 3\DL\Assignment\ass 3\fall_videos\mc_videos_test\chute01\cam1.avi: 448x640 1 person, 32.3ms
video 1/1 (frame 3/1562) C:\Users\sonnpm\OneDrive - UTS\Desktop\UTS\Semester 3\DL\Assignment\ass 3\fall_videos\mc_videos

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: ultralytics.engine.results.Keypoints object
 masks: None
 names: {0: 'person'}
 obb: None
 orig_img: array([[[16, 18, 20],
         [35, 37, 39],
         [42, 39, 56],
         ...,
         [ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0]],
 
        [[30, 32, 34],
         [36, 38, 40],
         [28, 25, 42],
         ...,
         [ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0]],
 
        [[34, 37, 37],
         [35, 38, 38],
         [29, 24, 38],
         ...,
         [ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0]],
 
        ...,
 
        [[31, 28, 29],
         [32, 29, 30],
         [31, 23, 38],
         ...,
         [ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0]],
 
        [[33, 28, 29],
         [34, 29, 30],
         [29, 21, 36],
         ...,
         [ 0,  0,  0],
         [ 0,  0,  0],
         [ 0, 

In [2]:
from ultralytics import YOLO
import pandas as pd
import os
import cv2

# Video to keypoints csv

# GENERATE keypoints here
model = YOLO("yolo11n-pose.pt")  # load an official model
input_folder = './fall_videos/mc_videos'

all_keypoints = []

def is_none_or_empty(x):
    return x is None or len(x) == 0

# normalize data according to the size of videos in different dataset
def normalize(list, width, height, offset):
    for i in range(offset, 51 + offset, 3):  # 17 keypoints, each keypoint is x, y and confidence
        list[i] = float(list[i]) / width
        list[i + 1] = float(list[i + 1]) / height

for root, dirs, files in os.walk(input_folder):
    # if "Videos" in root:
        for file in files:
            if file.endswith(".avi"):
                full_path = os.path.join(root, file)
                print("Processing {}".format(full_path))
                results = model.track(source=full_path, stream=True, verbose=False)

                folder_name = os.path.basename(root)  # "chute01"
                video_name = os.path.splitext(file)[0]  # "video"
                export_video_name = f"{folder_name}/{video_name}"  # "chute01/video"

                for frame_number, result in enumerate(results): # each is a frame
                    height, width = result.orig_img.shape[:2]
                    boxes = result.boxes  # Detections
                    keypoints = result.keypoints  # If using a pose model

                    if is_none_or_empty(boxes) or is_none_or_empty(keypoints):
                        continue

                    best_index = None
                    best_conf = 0.0

                    # inside a frame can have multiple people but only take the highest confidence
                    for i in range(len(boxes)):
                        conf = float(boxes.conf[i])
                        cls = int(boxes.cls[i])
                        if cls == 0 and conf > best_conf:
                            best_conf = conf
                            best_index = i

                    # print("best confidence: ", best_conf)

                    # keypoints.data is a torch.Tensor of shape (num_detections, num_keypoints, 3)
                    keypoints_tensor = keypoints.data[best_index]

                    keypoints_np = keypoints_tensor.cpu().detach().numpy()

                    # base_name = os.path.splitext(file)[0]

                    base_name = export_video_name
                    tag = f"{base_name}_{frame_number}"

                    flat = keypoints_np.flatten().tolist()
                    normalize(flat, width, height, 0)

                    # Extract bounding box coordinates
                    bbox = boxes[best_index].xyxy.squeeze(0).tolist()  # [xmin, ymin, xmax, ymax]

                    # Combine with flattened keypoints and tag
                    flat_with_tag = bbox + flat + [tag] # videoname_frame

                    # Store the result
                    all_keypoints.append(flat_with_tag)

df = pd.DataFrame(all_keypoints)
df.to_csv('./csv_data/mc_keypoints.csv', index=False, header=False)

Processing ./fall_videos/mc_videos\chute01\cam1.avi
Processing ./fall_videos/mc_videos\chute01\cam2.avi
Processing ./fall_videos/mc_videos\chute01\cam3.avi
Processing ./fall_videos/mc_videos\chute01\cam4.avi
Processing ./fall_videos/mc_videos\chute01\cam5.avi
Processing ./fall_videos/mc_videos\chute01\cam6.avi
Processing ./fall_videos/mc_videos\chute01\cam7.avi
Processing ./fall_videos/mc_videos\chute01\cam8.avi
Processing ./fall_videos/mc_videos\chute02\cam1.avi
Processing ./fall_videos/mc_videos\chute02\cam2.avi
Processing ./fall_videos/mc_videos\chute02\cam3.avi
Processing ./fall_videos/mc_videos\chute02\cam4.avi
Processing ./fall_videos/mc_videos\chute02\cam5.avi
Processing ./fall_videos/mc_videos\chute02\cam6.avi
Processing ./fall_videos/mc_videos\chute02\cam7.avi
Processing ./fall_videos/mc_videos\chute02\cam8.avi
Processing ./fall_videos/mc_videos\chute03\cam1.avi
Processing ./fall_videos/mc_videos\chute03\cam2.avi
Processing ./fall_videos/mc_videos\chute03\cam3.avi
Processing .

In [5]:
import pandas as pd

# Step 1: Read the label file (space-separated)
label_df = pd.read_csv('./csv_data/mc_label.csv', delim_whitespace=True, header=0, names=["video_name", "start_fall", "end_fall"])

# Convert label dataframe to a dict for quick lookup
label_dict = label_df.set_index('video_name')[['start_fall', 'end_fall']].to_dict(orient='index')

# Step 2: Read the data file (comma-separated)
data_df = pd.read_csv('./csv_data/mc_keypoints.csv', header=None)

# Assume last column contains 'videoname_framenumber'
last_col = data_df.columns[-1]

# Step 3: Extract video_name and frame_number
def extract_info(row):
    video_frame = row[last_col].strip()
    video_name, frame_str = video_frame.rsplit('_', 1)
    frame_num = int(frame_str)
    return pd.Series([video_name, frame_num])

data_df[['video_name', 'frame_number']] = data_df.apply(extract_info, axis=1)

# Step 4: Assign label based on falling range
def get_label(row):
    info = label_dict.get(row['video_name'], {'start_fall': -1, 'end_fall': -1})
    return 1 if info['start_fall'] != -1 and info['start_fall'] <= row['frame_number'] <= info['end_fall'] else 0

data_df['label'] = data_df.apply(get_label, axis=1)

# Step 5: Reorder columns: label first, then all original data
# Drop helper columns
data_df.drop(columns=['video_name', 'frame_number'], inplace=True)

# Move 'label' to the front
cols = ['label'] + [col for col in data_df.columns if col != 'label']
processed_df = data_df[cols]

# Step 6: Save to new CSV
processed_df.to_csv('mc_keypoints_labeled.csv', index=False, header=False)


C:\Users\sonnpm\AppData\Local\Temp\ipykernel_51196\2704384394.py:4: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  label_df = pd.read_csv('./csv_data/mc_label.csv', delim_whitespace=True, header=0, names=["video_name", "start_fall", "end_fall"])


In [6]:
print(processed_df.iloc[0].to_list())

# Print shape (rows, columns)
print("Shape:", processed_df.shape)

[0, 54.39403533935547, 21.05238342285156, 98.9665985107422, 147.21707153320312, 0.0, 0.0, 0.2791869342327118, 0.0, 0.0, 0.3492089509963989, 0.0, 0.0, 0.0494172163307666, 0.089092297024197, 0.0707973718643188, 0.8685168027877808, 0.0, 0.0, 0.101618118584156, 0.0946883731418185, 0.095867919921875, 0.984432578086853, 0.1130849944220648, 0.0910928169886271, 0.941720724105835, 0.0944540341695149, 0.1413047313690185, 0.9395712614059448, 0.1228462325202094, 0.1349337736765543, 0.5676352381706238, 0.0886841456095377, 0.181157382329305, 0.8227876424789429, 0.0, 0.0, 0.4278625547885895, 0.1085199779934353, 0.1726768016815185, 0.9887003302574158, 0.1201271798875596, 0.170335054397583, 0.977628767490387, 0.1092334429423014, 0.2335964997609456, 0.9734041094779968, 0.1149736616346571, 0.2315667470296224, 0.9390128254890442, 0.119143443637424, 0.2896625200907389, 0.929798662662506, 0.1180456373426649, 0.2839027086893717, 0.8770264387130737, 'chute01/cam1_0']
Shape: (214318, 57)


In [7]:
import pandas as pd

# Replace with your actual file name
file_path = './csv_data/le2i_keypoints_labeled.csv'

# Read the CSV
df = pd.read_csv(file_path)

# Print the shape
print("Shape:", df.shape)

# Optionally print the first few rows to inspect
print(df.head())

Shape: (69787, 57)
   0  20.215499877929688  104.03048706054688  50.93499755859375  \
0  0           20.215212          104.152710          50.887192   
1  0           20.202534          104.363121          50.784687   
2  0           20.220329          104.402504          50.817902   
3  0           20.181557          104.410995          50.758324   
4  0           20.187408          104.328461          50.781616   

   211.07791137695312  0.0  0.0.1  0.015844112262129784  0.0.2  0.0.3  ...  \
0          210.975464  0.0    0.0              0.018864    0.0    0.0  ...   
1          210.884918  0.0    0.0              0.017396    0.0    0.0  ...   
2          210.451324  0.0    0.0              0.015156    0.0    0.0  ...   
3          210.833328  0.0    0.0              0.014207    0.0    0.0  ...   
4          210.312317  0.0    0.0              0.014169    0.0    0.0  ...   

   0.0.28  0.0.29  0.15014342963695526  0.0.30  0.0.31  0.4452410340309143  \
0     0.0     0.0             0